In [ ]:
# Ch14-3-containers

In [ ]:
# Build and simple Docker container and push it to AWS ECR 

In [ ]:
# Import Libraries
import os
import boto3
import subprocess
import uuid
import logging

In [ ]:
# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

In [ ]:
# AWS Settings
AWS_REGION = 'us-east-1'  # Change to your preferred region
REPOSITORY_NAME = 'my-simple-app'  # Name for your ECR repository
IMAGE_TAG = 'latest'

# Create a unique ID for the build
BUILD_ID = str(uuid.uuid4())[:8]


In [ ]:
def create_dockerfile():
    """Create a simple Dockerfile for a Python application"""
    logger.info("Creating Dockerfile and application files...")
    
    # Create a directory for our app
    os.makedirs('app', exist_ok=True)
    
    # Create a simple Flask application
    with open('app/app.py', 'w') as f:
        f.write('''
from flask import Flask
app = Flask(__name__)

@app.route('/')
def hello():
    return "Hello from the Docker container!"

if __name__ == "__main__":
    app.run(host='0.0.0.0', port=5000)
''')
    
    # Create requirements.txt
    with open('app/requirements.txt', 'w') as f:
        f.write('flask==2.2.3\n')
    
    # Create Dockerfile
    with open('Dockerfile', 'w') as f:
        f.write('''
FROM python:3.9-slim

WORKDIR /app

COPY app/requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

COPY app/ .

EXPOSE 5000

CMD ["python", "app.py"]
''')
    
    logger.info("Dockerfile and application files created successfully")


In [ ]:
def build_docker_image():
    """Build the Docker image locally"""
    image_name = f"{REPOSITORY_NAME}:{IMAGE_TAG}"
    logger.info(f"Building Docker image: {image_name}")
    
    try:
        subprocess.run(
            ["docker", "build", "-t", image_name, "."],
            check=True,
            stdout=subprocess.PIPE,
            text=True
        )
        logger.info("Docker image built successfully")
        return image_name
    except subprocess.CalledProcessError as e:
        logger.error(f"Failed to build Docker image: {e}")
        raise


In [ ]:
def create_ecr_repository(ecr_client):
    """Create an ECR repository if it doesn't exist"""
    try:
        logger.info(f"Checking if repository {REPOSITORY_NAME} exists...")
        ecr_client.describe_repositories(repositoryNames=[REPOSITORY_NAME])
        logger.info(f"Repository {REPOSITORY_NAME} already exists")
    except ecr_client.exceptions.RepositoryNotFoundException:
        logger.info(f"Creating repository {REPOSITORY_NAME}...")
        response = ecr_client.create_repository(
            repositoryName=REPOSITORY_NAME,
            imageScanningConfiguration={'scanOnPush': True},
            encryptionConfiguration={'encryptionType': 'AES256'}
        )
        logger.info(f"Repository created: {response['repository']['repositoryUri']}")


In [ ]:
def get_ecr_login_command(ecr_client):
    """Get the ECR login command"""
    response = ecr_client.get_authorization_token()
    
    auth_data = response['authorizationData'][0]
    token = auth_data['authorizationToken']
    endpoint = auth_data['proxyEndpoint']
    
    return token, endpoint

In [ ]:
def tag_and_push_image(local_image_name, repository_uri):
    """Tag and push the Docker image to ECR"""
    logger.info(f"Tagging image as {repository_uri}")
    
    # Tag the image
    try:
        subprocess.run(
            ["docker", "tag", local_image_name, repository_uri],
            check=True,
            stdout=subprocess.PIPE,
            text=True
        )
        logger.info("Image tagged successfully")
    except subprocess.CalledProcessError as e:
        logger.error(f"Failed to tag image: {e}")
        raise
    
    # Push the image
    try:
        logger.info(f"Pushing image to {repository_uri}")
        subprocess.run(
            ["docker", "push", repository_uri],
            check=True,
            stdout=subprocess.PIPE,
            text=True
        )
        logger.info("Image pushed successfully")
    except subprocess.CalledProcessError as e:
        logger.error(f"Failed to push image: {e}")
        raise


In [ ]:
def login_to_ecr(region, endpoint):
    try:
        print(f"Logging into ECR at {endpoint}...")
        subprocess.run(
            f"aws ecr get-login-password --region {region} | docker login --username AWS --password-stdin {endpoint.replace('https://', '')}",
            shell=True,
            check=True
        )
        print("Logged in to ECR successfully.")
    except subprocess.CalledProcessError as e:
        print(f"Failed to log in to ECR: {e}")
        raise

In [ ]:
def main():
    """Main function to build and push a Docker image to ECR"""
    try:
        # Create sample application and Dockerfile
        create_dockerfile()
        
        # Build the Docker image locally
        local_image_name = build_docker_image()
        
        # Initialize boto3 ECR client
        logger.info(f"Connecting to AWS ECR in region {AWS_REGION}...")
        ecr_client = boto3.client('ecr', region_name=AWS_REGION)
        
        # Create the ECR repository if it doesn't exist
        create_ecr_repository(ecr_client)
        
        # Get the repository URI
        response = ecr_client.describe_repositories(repositoryNames=[REPOSITORY_NAME])
        repository_uri = response['repositories'][0]['repositoryUri']
        repository_uri_with_tag = f"{repository_uri}:{IMAGE_TAG}"
        
        # Get ECR authorization token
        token, endpoint = get_ecr_login_command(ecr_client)

        # Use the AWS CLI to log in to ECR
        logger.info("Logging in to ECR...")
        login_to_ecr(AWS_REGION, endpoint.replace("https://", ""))
                
        # Tag and push the image
        tag_and_push_image(local_image_name, repository_uri_with_tag)
        
        logger.info(f"Successfully built and pushed Docker image to: {repository_uri_with_tag}")
        logger.info("You can pull this image using:")
        logger.info(f"docker pull {repository_uri_with_tag}")
        
        return repository_uri_with_tag
    
    except Exception as e:
        logger.error(f"An error occurred: {e}")
        raise

if __name__ == "__main__":
    main()

In [ ]:
## End of Notebook ##